In [1]:
import folium
from IPython.display import IFrame
import pandas as pd

from folium.plugins import MarkerCluster
from IPython.display import IFrame

from ipywidgets import interact, interactive, FloatSlider, widgets, HBox, Label
from IPython.display import display
from ipywidgets import widgets
from ipywidgets import FloatSlider, widgets, RadioButtons

from IPython.display import display, clear_output, IFrame
import numpy as np
from sklearn.neighbors import NearestNeighbors

button1 = widgets.Button(description="Get CRL Location")
button2 = widgets.Button(description="Get Nearest Neighbors")
display(button1)
display(button2)  
# read the radiation data 
radiation = np.genfromtxt('radDataNew.csv',delimiter=',')
np.set_printoptions(precision=2)
Y = radiation[:,0:2] # extract firs two columns of the radiation data
detectorInput = Y[10] # this is the detector input. we take the 4th row of the data. Array [37.67, 140.07]
KNNInput=np.matrix([detectorInput])
#print(KNNInput)# this is KNN input [[  37.67  140.07   48.  ]]
#print (detectorInput)
#X= Y[0:2, :3] # this is KNN input point as multiple array. this is the third row of the data Y 
#print(X)
IFrame('urbana.html',width=700,height=600)
basemap = folium.Map(location=[37.54, 140.12],tiles='Stamen Terrain', zoom_start=10) 
fg=folium.FeatureGroup(name='my map')
def on_button1_clicked(b): 

    folium.CircleMarker([detectorInput[0],detectorInput[1]], radius=10, color='red',fill=True, fill_color='red', popup='Sensor Location',fill_opacity=0.7).add_to(basemap)
    
    basemap.save('urbana.html')
    display(IFrame('urbana.html',width=700,height=700))   
    # button clicked events
button1.on_click(on_button1_clicked)
    
def on_button2_clicked(b): 
    
    folium.CircleMarker([detectorInput[0],detectorInput[1]], radius=10, color='red',fill=True, fill_color='red', popup='Sensor Location',fill_opacity=0.7).add_to(basemap)
    knn = NearestNeighbors(n_neighbors=10, p=2, algorithm='ball_tree') # we generate 10 nearest neighbor distance matrix 
    knn.fit(Y)
    neighbors=knn.kneighbors(KNNInput, n_neighbors=4, return_distance=False) # query the nearst neighbor matrix and select the X's 6 nearest neighbors. 
    
    lenOfNeighbors=sum(len(x) for x in neighbors) # length of the nearest neighbors. In this case is 6 

    sumValue = 0
    for x in np.nditer(neighbors[0], order='F'):
        eachPointValue= radiation[x][2]
       
        sumValue += eachPointValue
        eachPointXcor = radiation[x][0]
        eachPointYcor = radiation[x][1]
       
        fg.add_child(folium.CircleMarker([eachPointXcor, eachPointYcor], radius=5, color='blue',fill=True, fill_color='blue',fill_opacity=0.7,popup='nearest K neighbors'))
        basemap.add_child(fg) 
    basemap.save('urbana.html')
    display(IFrame('urbana.html',width=700,height=700))
    
    meanValue = sumValue/ lenOfNeighbors # 21.833
    
    if meanValue>radiation[10,2]: 
        print('A radiation source is found!')

    else: 
        print('No radiation source is found!')
button2.on_click(on_button2_clicked)

    